In [1]:
%autosave 5

Autosaving every 5 seconds


In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import os 
import pickle
os.chdir('/content/drive/My Drive/MusicRecommendationSystem/Lalitha-Evaluation-1')
!ls
with open('weights22kwithfeatures.pickle', 'rb') as f:
    model_weights = pickle.load(f)

playlist_matrix = model_weights[0]
print(playlist_matrix.shape)
songs_matrix = model_weights[1]
print(songs_matrix.shape)

22kPlaylist+features+songindex+artistindex.csv	Evaluation-Lalitha.ipynb
Artists-weights22kwithfeatures.pickle		weights22kwithfeatures.pickle
(22740, 11)
(289796, 11)


In [4]:
''' Example code '''
import numpy as np
m = np.array([[1,2,3],[4,5,6],[7,8,9]])
c = np.array([0,1,2])
print(m * c[:, np.newaxis])

[[ 0  0  0]
 [ 4  5  6]
 [14 16 18]]


In [5]:
''' Weighting each feature 
 11 features : features = ['Danceability','Instrumentalness','Loudness','Speechiness','Valence','Energy','Mode','Tempo','Livenss','Key','Acousticness'] '''
weights=[0.75, 2, 2, 1, 1, 0.5, 0.05, 1, 0.5, 0.5, 0.75] 
songs_matrix_weighted = songs_matrix * weights[:np.newaxis]

In [6]:
def cosine_similarity(word_vectors,track_id,n):
  cosine_dict = {}
  word_list = []
  index = track_id -1 
  target = word_vectors[index,:]
  for i in range(0,word_vectors.shape[0]):
    curr = word_vectors[i]
    cos_sim = np.dot(curr,target)/(np.linalg.norm(curr)*np.linalg.norm(target))
    cosine_dict[i]=cos_sim
  dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True)
  for item in dist_sort:
    word_list.append((item[0]+1,item[1]))
  return word_list[0:n]

In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
train = pd.read_csv('22kPlaylist+features+songindex+artistindex.csv')
train.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,trackid,artist_name,track_name,duration_ms,album_name,pid,Danceability,Instrumentalness,Loudness,Speechiness,Valence,Energy,Mode,Tempo,Livenss,Key,Acousticness,Unnamed: 0.1.1.1.1,trackindex,artistindex
0,0,0,0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook,0,0.904,0.006970,-7.105,0.1210,0.810,0.813,0,125.461,0.0471,4,0.03110,NaN,18621,32614
1,1,1,1,1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,198800,In The Zone,0,0.774,0.025000,-3.914,0.1140,0.924,0.838,0,143.040,0.2420,5,0.02490,NaN,234387,6668
2,2,2,2,2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.000000,-6.583,0.2100,0.701,0.758,0,99.259,0.0598,2,0.00238,NaN,19991,5065
3,3,3,3,3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,267266,Justified,0,0.891,0.000234,-6.055,0.1400,0.818,0.714,0,100.972,0.0521,4,0.20200,NaN,43850,24706
4,4,4,4,4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,227600,Hot Shot,0,0.853,0.000000,-4.596,0.0713,0.654,0.606,1,94.759,0.3130,0,0.05610,NaN,66278,41903


In [8]:
input_song_names  = ['Agar Tum Saath Ho', 'Humse Pyaar Kar Le Tu']
song_ids = []
for song_name in input_song_names:
  rows = train.loc[train['track_name'] == song_name]
  try:
    song_id = rows['trackindex'].to_list()[0]
    song_ids.append(song_id)
  except:
    song_id = -1
  print(song_id)

print(song_ids) 
similar_song_ids = []
rec_ids = set()
for input_id in song_ids:
  print(input_id)
  get = cosine_similarity(songs_matrix_weighted,input_id,5)
  for rec_id in get:
    if rec_id not in rec_ids:
      rec_ids.add(rec_id)
      temp = []
      temp.append(rec_id[1])
      temp.append(rec_id[0])  
      temp.append(input_id)
      if temp[1] != temp[2]: #input song and recommended song shouldn't be the same
        similar_song_ids.append(temp)
similar_song_ids.sort()
print(similar_song_ids)

284753
152174
[284753, 152174]
284753
152174
[[0.9999822162055388, 281047, 284753], [0.9999823001514998, 12723, 284753], [0.9999828550635272, 22542, 284753], [0.9999861205798697, 144668, 284753], [0.9999881892154778, 185024, 152174], [0.9999886694435173, 161476, 152174], [0.9999890121535538, 4403, 152174], [0.9999928139524604, 131751, 152174]]


In [9]:
most_popular = ['HUMBLE', 'One Dance', 'Closer', 'Broccoli ft. Lil Yatchy', 'Congratulations', 'Bad and Boujee', 'Caroline', 'Bounce Back', 'Location', 'Mask Off', 'Despacito - Remix', "I'm the One", 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix']
output_list_1 = []
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  inp_song  = inp_rows['track_name'].tolist()[0]
  rows = train.loc[train['trackindex'] == track[1]]
  song_name = rows['track_name'].tolist()
  print(song_name[0])
  output_list_1.append(song_name[0])
if len(similar_song_ids) == 0:
  for song in most_popular:
    print(song)
    output_list_1.append(song)

Awuthule Kancane?
My Buddy
I Can't Help But Wonder Where I'm Bound
La Cosquillita
Mandinga - Live
The Zebrah
What Part of No
River Below


In [10]:
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  rec_rows = train.loc[train['trackindex'] == track[1]]
  # print(inp_rows.columns)
  req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness',
       'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
  inp_features = inp_rows[req_cols].values.tolist()[0]
  rec_features = rec_rows[req_cols].values.tolist()[0]
  for i in range(0, len(inp_features)):
    print(str(req_cols[i])+"\t"+str(inp_features[i])+"\t"+str(rec_features[i]))
  
  print("\n")

artist_name	Prokumar & Arijit Singh & Alka Yagnik	Mahlathini & The Mahotella Queens
album_name	In the Sky	Paris-Soweto
track_name	Agar Tum Saath Ho	Awuthule Kancane?
Danceability	0.557	0.8320000000000001
Instrumentalness	0.000379	0.0038
Loudness	-9.002	-7.258999999999999
Speechiness	0.0326	0.0538
Valence	0.401	0.8859999999999999
Energy	0.508	0.838
Mode	1	1
Tempo	122.999	116.92
Livenss	0.187	0.0808
Key	3	9
Acousticness	0.556	0.3720000000000001


artist_name	Prokumar & Arijit Singh & Alka Yagnik	Rico Richie
album_name	In the Sky	R.S.E.T. 3: Back 2 the Streets
track_name	Agar Tum Saath Ho	My Buddy
Danceability	0.557	0.7240000000000001
Instrumentalness	0.000379	0.0
Loudness	-9.002	-4.113
Speechiness	0.0326	0.0549
Valence	0.401	0.438
Energy	0.508	0.573
Mode	1	1
Tempo	122.999	113.088
Livenss	0.187	0.5379999999999999
Key	3	7
Acousticness	0.556	0.118


artist_name	Prokumar & Arijit Singh & Alka Yagnik	Dion & The Wanderers
album_name	In the Sky	The Road I'm On: A Retrospective
track_name	Agar T

**First Method of Evaluation**

For a given recommended song (**Sr**), find out the most similar input song  (**Si**)using cosine similarity as the metric

For 0 <= i < len(atrributes), precompute and store the ranges of each atrribute (**R_i**). Based on the ranges, define an interval **Delta_i** (the limit in which we want to call songs similar wrt that attribute)

For Sr and Si, for each attribute, calculate **diff_i = |inp_i - rec_i| / R_i**

Count the number of attributes which have diff_i <= Delta_i

If that number > 3 (or some preset threshold), then the recommmended song is considered as a **Good Recommendation**




In [11]:
attribute_ranges = []
numeric_cols = ['Danceability','Instrumentalness','Loudness','Speechiness','Valence','Energy','Mode','Tempo','Livenss','Key','Acousticness']
for attr in numeric_cols:
  print(attr)
  attr_index = train.columns.get_loc(attr)
  df_min = train[train[attr]==train[attr].min()].values.tolist()[0]
  print(df_min[attr_index])
  df_max = train[train[attr]==train[attr].max()].values.tolist()[0]
  print(df_max[attr_index])
  attribute_ranges.append(float(df_max[attr_index]-df_min[attr_index]))

print(attribute_ranges)

Danceability
0.0
0.991
Instrumentalness
0.0
0.9990000000000001
Loudness
-60.0
2.766
Speechiness
0.0
0.965
Valence
0.0
1.0
Energy
0.0
1.0
Mode
0
1
Tempo
0.0
249.703
Livenss
0.0
1.0
Key
0
11
Acousticness
0.0
0.996
[0.991, 0.9990000000000001, 62.766, 0.965, 1.0, 1.0, 1.0, 249.703, 1.0, 11.0, 0.996]


In [12]:
'''Returns True is the song is a good recommendation, False if it is not'''
def evaluate(item, attribute_ranges):
    reco = item[1]
    closest_inp = item[2]
    inp_rows = train.loc[train['trackindex'] == closest_inp]
    rec_rows = train.loc[train['trackindex'] == reco]
    # print(inp_rows.columns)
    req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness', 'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
    inp_features = inp_rows[req_cols].values.tolist()[0]
    rec_features = rec_rows[req_cols].values.tolist()[0]
    artist_index = train.columns.get_loc('artist_name')
    album_index = train.columns.get_loc('album_name')
    if inp_features[artist_index] == rec_features[artist_index]:
      return True
    if inp_features[album_index] == rec_features[album_index]:
      return True
    count = 0
    for i in range(3, len(inp_features)):
      val = float(abs(rec_features[i] - inp_features[i])/attribute_ranges[i-3])
      if val <= 0.2:
        count += 1
    if count >= 6:
      return True
    return False


count = 0
for item in similar_song_ids:
  if evaluate(item, attribute_ranges) == True:
    print("Good")
    count+=1
  else:
    print("Bad")

print(str(count) + " out of " + str(len(similar_song_ids)) + " songs are Good Recommendations")

Good
Good
Good
Good
Bad
Good
Good
Bad
6 out of 8 songs are Good Recommendations


Till now, the song recommendations were based on the representation of a song in terms of its playlist. (i.e the trained playlist-song matrix)

Now we'll generate another set of recommendations based on the representation of a song in terms of the Artist (trained artist-song matrix)

We'll combine both of the recommendations in some ratio to get our final set of recommended songs


In [13]:
with open('Artists-weights22kwithfeatures.pickle', 'rb') as f:
    model_weights = pickle.load(f)

artist_matrix = model_weights[0]
print(playlist_matrix.shape)
songs_matrix_2 = model_weights[1]
print(songs_matrix_2.shape)

(22740, 11)
(289796, 11)


In [14]:
input_song_names = ['Agar Tum Saath Ho', 'Humse Pyaar Kar Le Tu']
song_ids = []
for song_name in input_song_names:
  rows = train.loc[train['track_name'] == song_name]
  try:
    song_id = rows['trackindex'].to_list()[0]
    song_ids.append(song_id)
  except:
    song_id = -1
  print(song_id)

print(song_ids) 
similar_song_ids = []
rec_ids = set()
for input_id in song_ids:
  print(input_id)
  get = cosine_similarity(songs_matrix_2,input_id,5)
  for rec_id in get:
    if rec_id not in rec_ids:
      rec_ids.add(rec_id)
      temp = []
      temp.append(rec_id[1])
      temp.append(rec_id[0])  
      temp.append(input_id)
      if temp[1] != temp[2]: #input song and recommended song shouldn't be the same
        similar_song_ids.append(temp)
similar_song_ids.sort()
print(similar_song_ids)

284753
152174
[284753, 152174]
284753
152174
[[0.9997078, 155131, 284753], [0.9997328, 122174, 284753], [0.99981594, 47633, 284753], [0.99984443, 194814, 284753], [0.999911, 29824, 152174], [0.999911, 207117, 152174], [0.9999242, 241507, 152174], [0.99993265, 217636, 152174]]


In [15]:
output_list_2 = []
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  inp_song  = inp_rows['track_name'].tolist()[0]
  rows = train.loc[train['trackindex'] == track[1]]
  song_name = rows['track_name'].tolist()
  print(song_name[0])
  output_list_2.append(song_name[0])
if len(similar_song_ids) == 0:
  for song in most_popular:
    print(song)
    output_list_2.append(song)

Dupatta Tera Sat Rang Da
Hazey Jane I
Dawning
Hearts
No Me Compares
Drink My Rivers
Pretty Girl - Main (with rap)
Guelah Papyrus


In [16]:
for track in similar_song_ids:
  inp_rows = train.loc[train['trackindex'] == track[2]]
  rec_rows = train.loc[train['trackindex'] == track[1]]
  # print(inp_rows.columns)
  req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness',
       'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
  inp_features = inp_rows[req_cols].values.tolist()[0]
  rec_features = rec_rows[req_cols].values.tolist()[0]
  for i in range(0, len(inp_features)):
    print(str(req_cols[i])+"\t"+str(inp_features[i])+"\t"+str(rec_features[i]))
  
  print("\n")


artist_name	Prokumar & Arijit Singh & Alka Yagnik	Surjit Bindrakhia
album_name	In the Sky	Dupatta Tera Sat Rang Da
track_name	Agar Tum Saath Ho	Dupatta Tera Sat Rang Da
Danceability	0.557	0.467
Instrumentalness	0.000379	0.0
Loudness	-9.002	-9.038
Speechiness	0.0326	0.0916
Valence	0.401	0.948
Energy	0.508	0.708
Mode	1	0
Tempo	122.999	172.817
Livenss	0.187	0.598
Key	3	4
Acousticness	0.556	0.111


artist_name	Prokumar & Arijit Singh & Alka Yagnik	Nick Drake
album_name	In the Sky	Bryter Layter
track_name	Agar Tum Saath Ho	Hazey Jane I
Danceability	0.557	0.501
Instrumentalness	0.000379	0.000953
Loudness	-9.002	-19.106
Speechiness	0.0326	0.0336
Valence	0.401	0.155
Energy	0.508	0.215
Mode	1	1
Tempo	122.999	100.14
Livenss	0.187	0.129
Key	3	6
Acousticness	0.556	0.5770000000000001


artist_name	Prokumar & Arijit Singh & Alka Yagnik	Music For Alarm Apps
album_name	In the Sky	Ambient Tracks Vol. 1
track_name	Agar Tum Saath Ho	Dawning
Danceability	0.557	0.6829999999999999
Instrumentalness	0.000379	

In [17]:
'''Returns True is the song is a good recommendation, False if it is not'''
def evaluate(item, attribute_ranges):
    reco = item[1]
    closest_inp = item[2]
    inp_rows = train.loc[train['trackindex'] == closest_inp]
    rec_rows = train.loc[train['trackindex'] == reco]
    # print(inp_rows.columns)
    req_cols = ['artist_name', 'album_name', 'track_name', 'Danceability', 'Instrumentalness', 'Loudness', 'Speechiness', 'Valence', 'Energy', 'Mode', 'Tempo', 'Livenss', 'Key', 'Acousticness',]
    inp_features = inp_rows[req_cols].values.tolist()[0]
    rec_features = rec_rows[req_cols].values.tolist()[0]
    artist_index = train.columns.get_loc('artist_name')
    album_index = train.columns.get_loc('album_name')
    if inp_features[artist_index] == rec_features[artist_index]:
      return True
    if inp_features[album_index] == rec_features[album_index]:
      return True
    count = 0
    for i in range(3, len(inp_features)):
      val = float(abs(rec_features[i] - inp_features[i])/attribute_ranges[i-3])
      if val <= 0.2:
        count += 1
    if count >= 6:
      return True
    return False


count = 0
for item in similar_song_ids:
  if evaluate(item, attribute_ranges) == True:
    print("Good")
    count+=1
  else:
    print("Bad")

print(str(count) + " out of " + str(len(similar_song_ids)) + " songs are Good Recommendations")

Good
Good
Good
Good
Bad
Good
Good
Good
7 out of 8 songs are Good Recommendations


Final step: Combine both recommendations given by Artist and Playlist based representations


In [20]:
n1 = int(len(output_list_1)/2)-1
n2 = int(len(output_list_2)/2)-1
final_list = output_list_1[0:n1] + output_list_2[0:n2]
for song in final_list:
  print(song)

Awuthule Kancane?
My Buddy
I Can't Help But Wonder Where I'm Bound
Dupatta Tera Sat Rang Da
Hazey Jane I
Dawning
